## Classification

In [1]:
# Import stuff
import numpy as np
import scipy
import matplotlib
import matplotlib.pyplot as plt
from astropy.io import fits
import skymapper as skm
from code.plot_utils import plot_pretty
from astropy import units as u
from astropy.coordinates import SkyCoord
from matplotlib import rcParams
rcParams['font.family'] = 'serif'

%matplotlib inline
%pylab inline 
plot_pretty() 

Populating the interactive namespace from numpy and matplotlib


In [18]:
from Train_set import supervised

In [19]:
y_train, X_train = supervised()

In [20]:
# Import support vector machines
import sklearn
from sklearn.svm import SVC
svc_first = SVC(C=10,kernel='rbf')

In [21]:
# Fit the data now
svc_first.fit(X_train, y_train)

SVC(C=10, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto', kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

In [22]:
svc_first.score(X_train,y_train)

0.9411764705882353

In [23]:
REG_big = fits.open('Big_region.fits')

In [24]:
# Import everything 
# ========================================================
# ========================================================
# spread_model_i and its error
spread_model_i = REG_big[1].data['spread_model_i']
speraderr_model_i = REG_big[1].data['spreaderr_model_i']

# RA and DEC 
RA = REG_big[1].data['ra']
DEC = REG_big[1].data['dec']

# Image A/ Image B
IMAGE_A = REG_big[1].data['a_image']
IMAGE_B = REG_big[1].data['b_image']

# Magnitudes 
MAG_AUTO_G = REG_big[1].data['mag_auto_g']
MAG_AUTO_R = REG_big[1].data['mag_auto_r']
MAG_AUTO_I = REG_big[1].data['mag_auto_i']

# Flux radii - convert from pixels to arcseconds
FLUX_RADIUS_G = 0.263*REG_big[1].data['flux_radius_g']
FLUX_RADIUS_R = 0.263*REG_big[1].data['flux_radius_r']
FLUX_RADIUS_I = 0.263*REG_big[1].data['flux_radius_i'] 

# ================================================================================
# ================================================================================
# SURFACE BRIGHTNESSES

##Effective model surface brightness (SB) above background [mag/sq. arcmin]. 
#SB at the isophote which includes half of the flux from the model, above background
MU_EFF_G = REG_big[1].data['mu_eff_model_g']
MU_EFF_R = REG_big[1].data['mu_eff_model_r']
MU_EFF_I = REG_big[1].data['mu_eff_model_i']

#Peak surface brightness above background [mag/asec^2]
MU_MAX_G = REG_big[1].data['mu_max_g']
MU_MAX_R = REG_big[1].data['mu_max_r']
MU_MAX_I = REG_big[1].data['mu_max_i']

#Peak surface brightness above background  - using Model[mag/asec^2]
MU_MAX_MODEL_G = REG_big[1].data['mu_max_model_g']
MU_MAX_MODEL_R = REG_big[1].data['mu_max_model_r']
MU_MAX_MODEL_I = REG_big[1].data['mu_max_model_i']

#Mean surface brightness using the whole area inside the isophote used for MU_EFF_MODEL
MU_MEAN_G = REG_big[1].data['mu_mean_model_g']
MU_MEAN_R = REG_big[1].data['mu_mean_model_r']
MU_MEAN_I = REG_big[1].data['mu_mean_model_i']


# Define ellipticity 
ellipticity = 1.0 - IMAGE_B/IMAGE_A

print(len(RA))

6175361


In [25]:
# Define the cuts here 
# ====================================================

# Star-galaxy separation 
star_gal_cut = ((spread_model_i+(5.0/3.0)*speraderr_model_i)>0.007)

# Surface brightness cut
mu_eff_cut = (MU_EFF_G > 24.3)

# Color cuts 
col_1 = ((MAG_AUTO_G - MAG_AUTO_I)>(-0.1))
col_2 = ((MAG_AUTO_G - MAG_AUTO_I)<(1.4))
col_3 = ((MAG_AUTO_G - MAG_AUTO_R) > 0.7*(MAG_AUTO_G - MAG_AUTO_I) - 0.4)
col_4 = ((MAG_AUTO_G - MAG_AUTO_R) < 0.7*(MAG_AUTO_G - MAG_AUTO_I) + 0.4)
color_cuts = col_1&col_2&col_3&col_4

# Radius cut
radius_cut = ((FLUX_RADIUS_I > 2.5)&(FLUX_RADIUS_I < 20.0))

# Ellipticity cut
ell_cut = (ellipticity < 0.6)

# Summarize the final cut

Total_cut = star_gal_cut&mu_eff_cut&color_cuts&radius_cut&ell_cut

# RA and DEC
RA = RA[Total_cut]
DEC = DEC[Total_cut]

# Image A/ Image B
IMAGE_A = IMAGE_A[Total_cut]
IMAGE_B = IMAGE_B[Total_cut]

#Magnitudes 
MAG_AUTO_G = MAG_AUTO_G[Total_cut]
MAG_AUTO_R = MAG_AUTO_R[Total_cut]
MAG_AUTO_I = MAG_AUTO_I[Total_cut]

#Flux radii 
FLUX_RADIUS_G = FLUX_RADIUS_G[Total_cut]
FLUX_RADIUS_R = FLUX_RADIUS_R[Total_cut]
FLUX_RADIUS_I = FLUX_RADIUS_I[Total_cut]

# ================================================================================
# ================================================================================
# SURFACE BRIGHTNESSES

##Effective model surface brightness (SB) above background [mag/sq. arcmin]. 
#SB at the isophote which includes half of the flux from the model, above background
MU_EFF_G = MU_EFF_G[Total_cut]
MU_EFF_R = MU_EFF_R[Total_cut]
MU_EFF_I = MU_EFF_I[Total_cut]

#Peak surface brightness above background [mag/asec^2]
MU_MAX_G = MU_MAX_G[Total_cut]
MU_MAX_R = MU_MAX_R[Total_cut]
MU_MAX_I = MU_MAX_I[Total_cut]

#Peak surface brightness above background  - using Model[mag/asec^2]
MU_MAX_MODEL_G = MU_MAX_MODEL_G[Total_cut]
MU_MAX_MODEL_R = MU_MAX_MODEL_R[Total_cut]
MU_MAX_MODEL_I = MU_MAX_MODEL_I[Total_cut]

#Mean surface brightness using the whole area inside the isophote used for MU_EFF_MODEL
MU_MEAN_G = MU_MEAN_G[Total_cut]
MU_MEAN_R = MU_MEAN_R[Total_cut]
MU_MEAN_I = MU_MEAN_I[Total_cut]

# Ellipticity
ellipticity = ellipticity[Total_cut]

print(len(RA))

3597


In [26]:
# Define the colors
col_g_r = MAG_AUTO_G - MAG_AUTO_R
col_g_i = MAG_AUTO_G - MAG_AUTO_I
col_i_r = MAG_AUTO_G - MAG_AUTO_R



# Define the train data matrix of features
X_feat_test = np.zeros([len(RA),16])
X_feat_test[:,0] = MAG_AUTO_G;X_feat_test[:,1] = MAG_AUTO_R;X_feat_test[:,2] = MAG_AUTO_I
X_feat_test[:,3] = col_g_r;X_feat_test[:,4] = col_g_i;X_feat_test[:,5] = col_i_r
X_feat_test[:,6] = FLUX_RADIUS_G;X_feat_test[:,7] = FLUX_RADIUS_R;X_feat_test[:,8] = FLUX_RADIUS_I
X_feat_test[:,9] = MU_EFF_G;X_feat_test[:,10] = MU_EFF_R;X_feat_test[:,11] = MU_EFF_I
X_feat_test[:,12] = MU_MAX_G; X_feat_test[:,13] =  MU_MAX_R;X_feat_test[:,14] =  MU_MAX_I
X_feat_test[:,15] = ellipticity 

In [27]:
# Let's predict now
y_pred = svc_first.predict(X_feat_test)

In [28]:
RA_pos = RA[(y_pred==1)]
DEC_pos = DEC[(y_pred==1)]
print(len(RA_pos))

770


In [29]:
for i in range(100):
    print RA_pos[i],',',DEC_pos[i]

52.425021 , -32.678759
48.807117 , -33.398849
49.529478 , -30.530061
54.086329 , -29.86515
51.166767 , -33.21693
50.005169 , -33.81327
46.230174 , -31.06137
54.174329 , -34.71152
53.787343 , -31.059562
54.433597 , -32.431069
51.177303 , -25.497937
49.909634 , -26.408788
50.519609 , -30.380623
53.772832 , -33.428679
47.068365 , -26.701172
50.904153 , -28.393043
52.919467 , -32.841758
45.899512 , -26.405208
52.089586 , -33.825619
51.561213 , -25.985841
52.930947 , -28.753485
54.891178 , -25.609931
53.706881 , -28.143295
52.901214 , -32.846805
54.234678 , -25.864424
49.62028 , -25.492342
46.365126 , -27.753551
50.54091 , -28.251239
45.268216 , -25.350444
54.547306 , -33.451631
54.670864 , -27.50381
45.147381 , -25.263726
49.236144 , -30.404226
50.4859 , -29.164723
48.848912 , -34.267114
53.940358 , -30.072847
49.284952 , -32.568635
52.550191 , -26.640661
53.284111 , -30.265042
52.240735 , -27.330333
50.064494 , -33.355197
54.044822 , -25.359792
47.882061 , -29.151977
53.642431 , -26.05558

In [31]:
for i in range(100,200):
    print RA_pos[i],',',DEC_pos[i]

52.44778 , -26.870487
51.710554 , -32.678441
52.984471 , -32.216278
52.997589 , -28.328223
54.104047 , -25.716723
54.796883 , -31.62187
53.790327 , -27.537512
46.924205 , -34.187232
49.798171 , -32.917115
51.035971 , -33.825956
48.119571 , -33.561758
53.503605 , -27.838927
48.084278 , -32.836755
50.231698 , -26.67636
49.520323 , -25.82671
53.982302 , -34.828106
51.889764 , -26.880548
49.594851 , -28.538172
49.583188 , -27.088963
53.891581 , -34.804701
50.343169 , -29.535693
45.208773 , -32.53334
52.18862 , -25.329288
48.472394 , -34.817468
54.242841 , -26.20935
54.984666 , -30.496014
51.604441 , -26.461812
54.808989 , -33.531752
51.95912 , -30.787147
50.309479 , -26.099116
53.152317 , -34.105827
54.284031 , -34.73123
52.312031 , -34.921756
48.781857 , -25.920196
46.083325 , -25.927691
49.123596 , -26.623263
45.373509 , -28.656276
53.540657 , -34.030562
52.200198 , -31.123057
54.614685 , -25.096354
52.698783 , -34.426373
54.275727 , -34.98302
53.272029 , -29.730345
50.302568 , -30.39048